In [1]:
import os
from typing import List, Dict
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
import lightgbm as lgb
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import matplotlib.pyplot as plt

import seaborn as sns



In [2]:
# 파일 호출
data_path: str = "data"
train_df: pd.DataFrame = pd.read_csv(os.path.join(data_path, "train.csv")).assign(_type="train") # train 에는 _type = train 
test_df: pd.DataFrame = pd.read_csv(os.path.join(data_path, "test.csv")).assign(_type="test") # test 에는 _type = test
submission_df: pd.DataFrame = pd.read_csv(os.path.join(data_path, "test.csv")) # ID, target 열만 가진 데이터 미리 호출
df: pd.DataFrame = pd.concat([train_df, test_df], axis=0)
df


,ID,target,_type
0,2023-01-01 00:00:00,2.0,train
1,2023-01-01 01:00:00,1.0,train
2,2023-01-01 02:00:00,1.0,train
3,2023-01-01 03:00:00,1.0,train
4,2023-01-01 04:00:00,2.0,train
...,...,...,...
2787,2024-04-26 03:00:00,NaN,test
2788,2024-04-26 04:00:00,NaN,test
2789,2024-04-26 05:00:00,NaN,test
2790,2024-04-26 06:00:00,NaN,test


In [3]:
file_names: List[str] = [
    f for f in os.listdir(data_path) if f.startswith("HOURLY_") and f.endswith(".csv")
]
# 파일명 : 데이터프레임으로 딕셔너리 형태로 저장
file_dict: Dict[str, pd.DataFrame] = {
    f.replace(".csv", ""): pd.read_csv(os.path.join(data_path, f)) for f in file_names
}

for _file_name, _df in tqdm(file_dict.items()):
    # 열 이름 중복 방지를 위해 {_file_name.lower()}_{col.lower()}로 변경, datetime 열을 ID로 변경
    _rename_rule = {
        col: f"{_file_name.lower()}_{col.lower()}" if col != "datetime" else "ID"
        for col in _df.columns
    }
    _df = _df.rename(_rename_rule, axis=1)
    df = df.merge(_df, on="ID", how="left")

100%|██████████| 107/107 [00:02<00:00, 47.69it/s]


In [4]:
cols_dict: Dict[str, str] = {
    "ID": "ID",
    "target": "target",
    "_type": "_type",
    "hourly_market-data_coinbase-premium-index_coinbase_premium_gap": "coinbase_premium_gap",
    "hourly_market-data_coinbase-premium-index_coinbase_premium_index": "coinbase_premium_index",
    "hourly_market-data_funding-rates_all_exchange_funding_rates": "funding_rates",
    "hourly_market-data_liquidations_all_exchange_all_symbol_long_liquidations": "long_liquidations",
    "hourly_market-data_liquidations_all_exchange_all_symbol_long_liquidations_usd": "long_liquidations_usd",
    "hourly_market-data_liquidations_all_exchange_all_symbol_short_liquidations": "short_liquidations",
    "hourly_market-data_liquidations_all_exchange_all_symbol_short_liquidations_usd": "short_liquidations_usd",
    "hourly_market-data_open-interest_all_exchange_all_symbol_open_interest": "open_interest",
    "hourly_market-data_taker-buy-sell-stats_all_exchange_taker_buy_ratio": "buy_ratio",
    "hourly_market-data_taker-buy-sell-stats_all_exchange_taker_buy_sell_ratio": "buy_sell_ratio",
    "hourly_market-data_taker-buy-sell-stats_all_exchange_taker_buy_volume": "buy_volume",
    "hourly_market-data_taker-buy-sell-stats_all_exchange_taker_sell_ratio": "sell_ratio",
    "hourly_market-data_taker-buy-sell-stats_all_exchange_taker_sell_volume": "sell_volume",
    "hourly_network-data_transactions-count_transactions_count_total" : 'transaction_count',
    'hourly_network-data_hashrate_hashrate' : "hashrate",
     "hourly_network-data_addresses-count_addresses_count_active": "active_count",
    "hourly_network-data_addresses-count_addresses_count_receiver": "receiver_count",
    "hourly_network-data_addresses-count_addresses_count_sender": "sender_count",
}
# HOURLY_MARKET-DATA_PRICE-OHLCV_ALL_EXCHANGE_SPOT_BTC_USD

df = df[cols_dict.keys()].rename(cols_dict, axis=1)
df.shape


(11552, 21)

In [5]:
df = df.assign(
    liquidation_diff=df["long_liquidations"] - df["short_liquidations"],
    liquidation_usd_diff=df["long_liquidations_usd"] - df["short_liquidations_usd"],
    volume_diff=df["buy_volume"] - df["sell_volume"],
    liquidation_diffg=np.sign(df["long_liquidations"] - df["short_liquidations"]),
    liquidation_usd_diffg=np.sign(df["long_liquidations_usd"] - df["short_liquidations_usd"]),
    volume_diffg=np.sign(df["buy_volume"] - df["sell_volume"]),
    buy_sell_volume_ratio=df["buy_volume"] / (df["sell_volume"] + 1),
    
)
df['buy_volume_ma_3h'] = df['buy_volume'].rolling(window=3).mean()
df['buy_volume_std_3h'] = df['buy_volume'].rolling(window=3).std()

df['funding_rates_ma_3h'] = df['funding_rates'].rolling(window=3).mean()
df['funding_rates_std_3h'] = df['funding_rates'].rolling(window=3).std()

df['price_close_pct_change_1h'] = 100*(df['coinbase_premium_gap']-df['coinbase_premium_gap'].shift(1))/(df['coinbase_premium_gap'].shift(1)+1)
df['price_close_pct_change_3h'] = 100*(df['coinbase_premium_gap']-df['coinbase_premium_gap'].shift(3))/(df['coinbase_premium_gap'].shift(3)+1)

df['buy_volume_pct_change_1h'] = 100*(df['buy_volume']-df['buy_volume'].shift(1))/(df['buy_volume'].shift(1)+100)
df['buy_volume_pct_change_3h'] = 100*(df['buy_volume']-df['buy_volume'].shift(3))/(df['buy_volume'].shift(3)+100)

df['liquidation_diff_pct_change_3h'] = 100*(df['liquidation_diff']-df['liquidation_diff'].shift(3))/(df['liquidation_diff'].shift(3)+1)

df['buy_sell_ratio_pct_change_3h'] = df['buy_sell_ratio'].pct_change(periods=3)
df['is_buy_dominant'] = (df['buy_sell_ratio'] > 1.0).astype(int)

df['volume_diff_pct_change_12h'] = df['volume_diff'].pct_change(periods=12)
df['volume_diff_pct_change_6h'] = df['volume_diff'].pct_change(periods=6)
df['volume_diff_pct_change_additive']=df['volume_diff_pct_change_6h']+df['volume_diff_pct_change_12h'] # star
df['mean_7days'] = df['volume_diff'].rolling(window=168).mean()  # 7 days if hourly data
df['mean_1day'] = df['volume_diff'].rolling(window=24).mean()    # 1 day if hourly data
df['slope_7days'] = df['mean_7days'].diff()
df['slope_1day'] = df['mean_1day'].diff()
df['slope_change']=df['slope_7days']-df['slope_1day'] # star
df.drop(columns=['mean_7days', 'mean_1day'], inplace=True)




/var/folders/zk/056hcq1553j8j8d2gphz8ckm0000gn/T/ipykernel_92057/1801033282.py:25: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df['buy_sell_ratio_pct_change_3h'] = df['buy_sell_ratio'].pct_change(periods=3)
/var/folders/zk/056hcq1553j8j8d2gphz8ckm0000gn/T/ipykernel_92057/1801033282.py:28: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df['volume_diff_pct_change_12h'] = df['volume_diff'].pct_change(periods=12)
/var/folders/zk/056hcq1553j8j8d2gphz8ckm0000gn/T/ipykernel_92057/1801033282.py:29: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a 

In [ ]:

def Null_fill(
    df: pd.DataFrame,
    method: str = 'mean'  # Default to 'mean'
) -> pd.DataFrame:
    """
    Fills missing values in the DataFrame based on the specified method.

    Parameters:
    - df: The DataFrame to fill missing values in.
    - method: The method to use for filling missing values. Options are 'mean', 'median', 'mode'.

    Returns:
    - df: The DataFrame with filled missing values.
    """
    # Loop through all columns in the DataFrame
    df_train=df[df['_type']=='train']
    for col in df.columns:
        # Check if the column contains any NaN values
        if df[col].isna().sum() != 0:
            # Choose the appropriate method for filling missing values
            if method == 'mean':
                val = df_train[col].mean()
            elif method == 'median':
                val = df_train[col].median()
            elif method == 'mode':
                val = df_train[col].mode()[0]  # Use the first mode if multiple
            else:
                raise ValueError(f"Invalid method: {method}. Choose 'mean', 'median', or 'mode'.")
            
            # Fill NaN values in the column
            df[col] = df[col].fillna(val)
    
    return df